In [69]:
import os.path
import random
from matplotlib import pyplot, image
from pathlib import Path
import numpy as np
from PIL import Image
import dask.dataframe as dd
from dask.distributed import Client
from dask.distributed import get_worker
from dask.distributed import progress
from fabric import Connection, Config

In [70]:
client = Client("tcp://128.110.217.91:8786")

/users/prsridha/.local/lib/python3.8/site-packages/distributed/client.py:1265: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | client | scheduler | workers |
+---------+--------+-----------+---------+
| blosc   | 1.7.0  | 1.7.0     | None    |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [71]:
client

<Client: 'tcp://128.110.217.91:8786' processes=2 threads=32, memory=125.38 GiB>

In [4]:
DATA_URL = "metadata.csv"
feature_names = ["image_path", "label"]
dtypes = {'image_path': np.str, 'label': np.int16}
df0 = dd.read_csv(DATA_URL, names=feature_names, dtype=dtypes)
df = df0.sample(frac=0.001)

/tmp/ipykernel_329363/3867680041.py:3: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtypes = {'image_path': np.str, 'label': np.int16}


In [5]:
df.head()

,image_path,label
2953,data/trainingSet/trainingSet/0/img_18803.jpg,0
29198,data/trainingSet/trainingSet/6/img_32535.jpg,6
32302,data/trainingSet/trainingSet/7/img_21763.jpg,7
28964,data/trainingSet/trainingSet/6/img_30048.jpg,6
18157,data/trainingSet/trainingSet/4/img_41403.jpg,4


In [6]:
user = "prsridha"
host = "ms0921.utah.cloudlab.us"
pem_path = "./cloudlab.pem"
connect_kwargs = {"key_filename":pem_path}
conn = Connection(host, user=user, connect_kwargs=connect_kwargs)

def pull(from_path, to_path):
    to_path_dir = "/".join(to_path.split("/")[:-1])
    Path(to_path_dir).mkdir(parents=True, exist_ok=True)
    result = conn.get(from_path, to_path)
    print("Pulled from {0.remote} to {0.local}".format(result))

In [72]:
def initialize_workers(client):
    """Initialize workers (get worker IPs)"""
    worker_id_ip_dict = {}
    all_worker_details = client.scheduler_info()['workers']
    i = 0
    for ip in all_worker_details:
        # set the mapping between worker ID and worker IP
        worker_id_ip_dict[i] = str(ip)
        i += 1
    return worker_id_ip_dict
workers = initialize_workers(client)

In [61]:
shuffled_df = df.sample(frac=1)
shuffled_df.head()

,image_path,label
27125,data/trainingSet/trainingSet/6/img_10461.jpg,6
26527,data/trainingSet/trainingSet/6/img_27011.jpg,6
40677,data/trainingSet/trainingSet/9/img_40807.jpg,9
41660,data/trainingSet/trainingSet/9/img_11783.jpg,9
32302,data/trainingSet/trainingSet/7/img_21763.jpg,7


In [62]:
new_df = shuffled_df.repartition(npartitions=2)

In [57]:
part1 = new_df.partitions[0]
part2 = new_df.partitions[1]

In [63]:
n1 = list(part1.image_path.compute())
n2 = list(part2.image_path.compute())
with open("worker1_list.txt", "w") as f:
    f.write("\n".join(n1))
with open("worker2_list.txt", "w") as f:
    f.write("\n".join(n2))

In [73]:
workers

{0: 'tcp://128.110.217.112:40287', 1: 'tcp://128.110.217.73:37225'}

In [64]:
def preprocess_row(row):
    return str(row["image_path"]) + ";" + str(row["label"])

In [65]:
def preprocess_col(path):
    new_path = "/users/prsridha/" + path
#     pull(new_path, new_path)
    
    im = Image.open(str(new_path))
    pixels = list(im.getdata())
    return sum(pixels)

In [66]:
new_col1 = part1.image_path.map_partitions(lambda x: x.apply(lambda y: preprocess_col(str(y))), meta=('new_col1', int))
new_col2 = part2.image_path.map_partitions(lambda x: x.apply(lambda y: preprocess_col(str(y))), meta=('new_col2', int))

In [67]:
new_col1.compute(workers=workers[0])

FileNotFoundError: [Errno 2] No such file or directory: '/users/prsridha/data/trainingSet/trainingSet/6/img_10461.jpg'

In [110]:
workers

{0: 'tcp://128.110.217.112:37319', 1: 'tcp://128.110.217.73:37365'}

In [74]:
client.scheduler_info()['workers']

{'tcp://128.110.217.112:40287': {'type': 'Worker',
  'id': 'tcp://128.110.217.112:40287',
  'host': '128.110.217.112',
  'resources': {},
  'local_directory': '/users/prsridha/dask-worker-space/worker-ijvyfc31',
  'name': 'tcp://128.110.217.112:40287',
  'nthreads': 16,
  'memory_limit': 67311730688,
  'last_seen': 1644989164.6080651,
  'services': {'dashboard': 41315},
  'metrics': {'executing': 0,
   'in_memory': 0,
   'ready': 0,
   'in_flight': 0,
   'bandwidth': {'total': 100000000, 'workers': {}, 'types': {}},
   'spilled_nbytes': 0,
   'cpu': 4.0,
   'memory': 121909248,
   'time': 1644989164.107899,
   'read_bytes': 2622.3946217062794,
   'write_bytes': 132.12064506306447,
   'read_bytes_disk': 0.0,
   'write_bytes_disk': 0.0,
   'num_fds': 24},
  'nanny': 'tcp://128.110.217.112:38577'},
 'tcp://128.110.217.73:37225': {'type': 'Worker',
  'id': 'tcp://128.110.217.73:37225',
  'host': '128.110.217.73',
  'resources': {},
  'local_directory': '/users/prsridha/dask-worker-space/wo

In [ ]:
conn.close()